In [1]:
import os 
import json
import vertexai
import base64
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/giorady/Documents/Work/Datalabs/PythonDev/key.json"
credentials = service_account.Credentials.from_service_account_file('/Users/giorady/Documents/Work/Datalabs/PythonDev/klg-workshop/key_kawan_lama.json')

In [3]:
vertexai.init(project="gcp-kl-fin-genai-poc", location="asia-southeast1", credentials=credentials)
model = GenerativeModel("gemini-1.5-pro-preview-0409")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 32,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

In [4]:
# File from local machine 
# with open("/Users/giorady/Documents/Work/Datalabs/PythonDev/klg-workshop/pdf_dummy_klg.pdf", "rb") as pdf_file:
#     encoded_string = base64.b64encode(pdf_file.read())

with open("/Users/giorady/Documents/Work/Datalabs/PythonDev/klg-workshop/Order Form Example.pdf", "rb") as pdf_file:
    encoded_string = base64.b64encode(pdf_file.read())

# File from google cloud storage 
# uri = "gs://kawan_lama_workshop_gio/Order Form Example.pdf"

In [5]:
document = Part.from_data(
mime_type="application/pdf",
data = base64.b64decode(encoded_string)
)

# document = Part.from_uri(
# mime_type="application/pdf",
# uri=uri
# )

In [6]:
prompt = """Extract Tenant Name followed by the Company Legal Name, Unit Ruang Sewa, Periode Sewa (convert from months to year) from the file and give the output in a format like this without any "json" string indicator in the beginning of your response. Just follow the format:
"nama_penyewa": "XXXX - PT.XXXX",
"lokasi": "x, x, x",
"masa_sewa": "x years"
"""

# prompt = """Extract Reseller Street Address, Reseller Name, and the duration term of the products ordered from the file and give the output in a format like this:
# {
# "street_address": "XXXX",
# "name": "XXXX",
# "term": "XXXX"
# }
# """

In [7]:
responses = model.generate_content(
    [document,prompt],
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

Unauthenticated: 401 Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.

In [ ]:
vertexai.init(project="gcp-kl-fin-genai-poc", location="asia-southeast1", credentials=credentials)
model = GenerativeModel("gemini-1.5-pro-preview-0409")

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 32,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

In [ ]:
# responses.candidates[0].content

In [ ]:
result = responses.text
print(result)

{
"street_address": "Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia",
"name": "Mendi Susanto",
"term": "12 Months"
}


In [ ]:
result_json = json.loads(result)
print(result_json)

{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia', 'name': 'Mendi Susanto', 'term': '12 Months'}


In [ ]:
result_json

{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia',
 'name': 'Mendi Susanto',
 'term': '12 Months'}

In [ ]:
json_rows=[result_json]
print(json_rows)

[{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia', 'name': 'Mendi Susanto', 'term': '12 Months'}]


In [ ]:
json_rows

[{'street_address': 'Wisma Nugra Santana, 2nd Fl. Jl. Jend. Sudirman Kav 7-8 Jakarta\n10220 Indonesia',
  'name': 'Mendi Susanto',
  'term': '12 Months'}]

In [ ]:
table_id = "gcp-kl-fin-genai-poc.db_TenantLW.resultTable"
client = bigquery.Client()
job = client.load_table_from_json(json_rows, table_id)
job.result()

LoadJob<project=datalabs-int-bigdata, location=asia-southeast2, id=15b8c619-c746-41fa-ad0d-b546fb01dd9d>